In [ ]:
# ==========================================
# 步骤 1: 安装依赖 & 挂载 Drive
# ==========================================
import os
from google.colab import drive

# 1. 安装 Whisper (支持 turbo) 和 ffmpeg
print("⏳ 正在安装 Whisper 及必要组件...")
!pip install git+https://github.com/openai/whisper.git -q
!sudo apt update && sudo apt install ffmpeg -q

# 2. 挂载 Google Drive
print("📂 正在挂载 Google Drive...")
drive.mount('/content/drive')
print("✅ 环境准备完成！请继续运行下一个单元格。")

In [ ]:
import os
import shutil
import time
import re
import requests
import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from google.colab import runtime

# ================= 配置区域 (修改这里) =================

# 1. 文件与路径
AUDIO_FILENAME = "example.mp3"  # 文件名
DRIVE_FOLDER = "/content/drive/MyDrive/podcast"          # Drive 文件夹路径

# 2. 翻译 API
API_KEY = "sk-**********************"                    # 您的 API Key
BASE_URL = "https://******************/v1"               # API 地址
MODEL_NAME = "gpt-4.1-mini"                              # 模型名称

# 3. 参数
MAX_WORKERS = 5   # 翻译并发数
MAX_RETRIES = 2   # 翻译重试次数

# ======================================================

def format_time(seconds):
    """将秒数转换为 时:分:秒 格式"""
    return str(datetime.timedelta(seconds=int(seconds)))

def main_logic():
    # --- 记录开始时间 ---
    start_time = time.time()
    print(f"⏱️ 任务开始时间: {time.strftime('%H:%M:%S', time.localtime(start_time))}")

    # 路径处理
    audio_path = os.path.join(DRIVE_FOLDER, AUDIO_FILENAME)
    base_name = os.path.splitext(AUDIO_FILENAME)[0]
    srt_en_name = f"{base_name}_en.srt"
    srt_final_name = f"{base_name}.srt"

    local_srt_en = f"/content/{srt_en_name}"
    local_srt_final = f"/content/{srt_final_name}"
    drive_srt_en = os.path.join(DRIVE_FOLDER, srt_en_name)
    drive_srt_final = os.path.join(DRIVE_FOLDER, srt_final_name)

    # ---------------------------
    # 阶段 1: Whisper 转录
    # ---------------------------
    if not os.path.exists(audio_path):
        print(f"❌ 错误: 找不到文件 {audio_path}")
        return

    print(f"\n🚀 [1/2] 正在进行 Whisper 转录 (Turbo模型)...")
    # 执行命令
    exit_code = os.system(f'whisper "{audio_path}" --model turbo --language English --output_format srt --verbose False --output_dir /content/')

    # 处理文件名
    default_output = f"/content/{base_name}.srt"
    if os.path.exists(default_output):
        if os.path.exists(local_srt_en): os.remove(local_srt_en)
        os.rename(default_output, local_srt_en)
        shutil.copy2(local_srt_en, drive_srt_en)
        print(f"✅ 英文 SRT 已保存: {srt_en_name}")
    else:
        print("❌ 转录失败，停止后续步骤。")
        return

    # ---------------------------
    # 阶段 2: API 翻译
    # ---------------------------
    print(f"\n🚀 [2/2] 正在进行并发翻译...")

    # 工具函数定义
    def parse_srt(content):
        pattern = re.compile(r'(\d+)\n(\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3})\n(.*?)(?=\n\n|\Z)', re.DOTALL)
        return [{'index': m[0], 'time': m[1], 'text': m[2].strip()} for m in pattern.findall(content + '\n\n')]

    def build_srt(subs):
        return "\n".join([f"{s['index']}\n{s['time']}\n{s['text']}\n" for s in subs])

    def translate_text(text, idx, is_retry=False):
        time.sleep(0.2 if not is_retry else 0.5)
        if not text.strip(): return idx, ""
        try:
            resp = requests.post(
                f"{BASE_URL}/chat/completions",
                headers={"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"},
                json={
                    "model": MODEL_NAME,
                    "messages": [{"role": "system", "content": "Translate the subtitle to Simplified Chinese. Output ONLY translation."}, {"role": "user", "content": text}],
                    "temperature": 0.3
                }, timeout=30
            )
            resp.raise_for_status()
            return idx, resp.json()['choices'][0]['message']['content'].strip()
        except:
            return idx, None

    # 读取与翻译逻辑
    with open(local_srt_en, 'r', encoding='utf-8') as f:
        subtitles = parse_srt(f.read())

    total_lines = len(subtitles)
    translation_map = {}

    # 并发执行
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(translate_text, sub['text'], i): i for i, sub in enumerate(subtitles)}
        finished = 0
        for f in as_completed(futures):
            finished += 1
            if finished % 20 == 0: print(f"   进度: {finished}/{total_lines}")
            idx, res = f.result()
            if res: translation_map[idx] = res

    # 重试逻辑
    failed = [i for i, s in enumerate(subtitles) if i not in translation_map and s['text'].strip()]
    if failed:
        print(f"🔄 正在重试 {len(failed)} 条失败项目...")
        with ThreadPoolExecutor(max_workers=2) as executor:
            for f in as_completed({executor.submit(translate_text, subtitles[i]['text'], i, True): i for i in failed}):
                idx, res = f.result()
                if res: translation_map[idx] = res

    # 合并
    success_count = 0
    for i, sub in enumerate(subtitles):
        zh = translation_map.get(i, "")
        if zh:
            sub['text'] = f"{sub['text']}\n{zh}"
            success_count += 1

    # 保存
    with open(local_srt_final, 'w', encoding='utf-8') as f: f.write(build_srt(subtitles))
    shutil.copy2(local_srt_final, drive_srt_final)

    print(f"\n✅ 翻译完成！成功率: {success_count}/{total_lines}")
    print(f"💾 最终文件已保存至 Drive: {srt_final_name}")

    # --- 统计总耗时 ---
    end_time = time.time()
    duration = end_time - start_time
    print(f"\n{'='*30}")
    print(f"📊 统计报告")
    print(f"🕒 总耗时: {format_time(duration)}")
    print(f"{'='*30}")

# 执行主逻辑
try:
    main_logic()
except Exception as e:
    print(f"❌ 程序发生错误: {e}")
finally:
    # ---------------------------
    # 自动关闭运行时
    # ---------------------------
    print("\n👋 任务结束。")
    print("⏳ 5秒后将自动断开 Google Colab 连接...")
    time.sleep(5)
    print("🔌 正在断开运行时 (Runtime)...")
    runtime.unassign()